In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sephora-products-and-skincare-reviews/product_info.csv
/kaggle/input/sephora-products-and-skincare-reviews/reviews_250_500.csv
/kaggle/input/sephora-products-and-skincare-reviews/reviews_1500_end.csv
/kaggle/input/sephora-products-and-skincare-reviews/reviews_1000_1500.csv
/kaggle/input/sephora-products-and-skincare-reviews/reviews_500_750.csv
/kaggle/input/sephora-products-and-skincare-reviews/reviews_750_1000.csv
/kaggle/input/sephora-products-and-skincare-reviews/reviews_0_250.csv


In [2]:
df_products = pd.read_csv('/kaggle/input/sephora-products-and-skincare-reviews/product_info.csv')

In [3]:
df_products.shape
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8494 entries, 0 to 8493
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_id          8494 non-null   object 
 1   product_name        8494 non-null   object 
 2   brand_id            8494 non-null   int64  
 3   brand_name          8494 non-null   object 
 4   loves_count         8494 non-null   int64  
 5   rating              8216 non-null   float64
 6   reviews             8216 non-null   float64
 7   size                6863 non-null   object 
 8   variation_type      7050 non-null   object 
 9   variation_value     6896 non-null   object 
 10  variation_desc      1250 non-null   object 
 11  ingredients         7549 non-null   object 
 12  price_usd           8494 non-null   float64
 13  value_price_usd     451 non-null    float64
 14  sale_price_usd      270 non-null    float64
 15  limited_edition     8494 non-null   int64  
 16  new   

In [4]:
df_products.isna().sum(axis=0)

product_id               0
product_name             0
brand_id                 0
brand_name               0
loves_count              0
rating                 278
reviews                278
size                  1631
variation_type        1444
variation_value       1598
variation_desc        7244
ingredients            945
price_usd                0
value_price_usd       8043
sale_price_usd        8224
limited_edition          0
new                      0
online_only              0
out_of_stock             0
sephora_exclusive        0
highlights            2207
primary_category         0
secondary_category       8
tertiary_category      990
child_count              0
child_max_price       5740
child_min_price       5740
dtype: int64

In [5]:
df_products.groupby(['brand_name'])['product_name'].count().sort_values(ascending=False).head(10)

brand_name
SEPHORA COLLECTION         352
CLINIQUE                   179
Dior                       136
tarte                      131
NEST New York              115
Bumble and bumble          110
Kérastase                  108
TOM FORD                   100
Charlotte Tilbury           99
Anastasia Beverly Hills     95
Name: product_name, dtype: int64

In [6]:
#popular product_category in each brand

tmp = pd.DataFrame(df_products.groupby(['brand_name','primary_category'])['product_id'].count().reset_index())
tmp['product_rank'] = tmp.groupby(['brand_name'])['product_id'].rank(method = 'min',ascending = False)
tmp[tmp.product_rank == 1]

,brand_name,primary_category,product_id,product_rank
0,19-69,Fragrance,14,1.0
1,54 Thrones,Bath & Body,4,1.0
2,ABBOTT,Fragrance,13,1.0
3,AERIN,Fragrance,22,1.0
6,ALTERNA Haircare,Hair,45,1.0
...,...,...,...,...
618,philosophy,Bath & Body,1,1.0
619,rms beauty,Makeup,11,1.0
620,shu uemura,Hair,43,1.0
621,stila,Makeup,10,1.0


In [7]:
#popular product in each brand
tmp1 = df_products[['brand_name','product_name','loves_count']]
tmp1['rank']= tmp1.groupby(['brand_name'])['loves_count'].rank(method='min',ascending=False)
tmp1

/tmp/ipykernel_20/3412678004.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['rank']= tmp1.groupby(['brand_name'])['loves_count'].rank(method='min',ascending=False)


,brand_name,product_name,loves_count,rank
0,19-69,Fragrance Discovery Set,6320,1.0
1,19-69,La Habana Eau de Parfum,3827,2.0
2,19-69,Rainbow Bar Eau de Parfum,3253,3.0
3,19-69,Kasbah Eau de Parfum,3018,4.0
4,19-69,Purple Haze Eau de Parfum,2691,5.0
...,...,...,...,...
8489,Yves Saint Laurent,Couture Clutch Eyeshadow Palette,2790,61.0
8490,Yves Saint Laurent,L'Homme Eau de Parfum,2319,62.0
8491,Yves Saint Laurent,Mon Paris Eau de Parfum Gift Set,1475,63.0
8492,Yves Saint Laurent,Y Eau de Parfum Gift Set,840,64.0


In [8]:
#Average price of each product category
tmp = df_products[['brand_name','primary_category','secondary_category','tertiary_category','price_usd']]
tmp['min_price']=tmp.groupby(['primary_category','secondary_category','tertiary_category'])['price_usd'].min()
tmp['max_price']=tmp.groupby(['primary_category','secondary_category','tertiary_category'])['price_usd'].max()
tmp['mean_price']=tmp.groupby(['primary_category','secondary_category','tertiary_category'])['price_usd'].mean()
tmp

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pandas/core/frame.py", line 12020, in _reindex_for_setitem
    reindexed_value = value.reindex(index)._values
  File "/opt/conda/lib/python3.10/site-packages/pandas/core/series.py", line 5094, in reindex
    return super().reindex(**kwargs)
  File "/opt/conda/lib/python3.10/site-packages/pandas/core/generic.py", line 5289, in reindex
    return self._reindex_axes(
  File "/opt/conda/lib/python3.10/site-packages/pandas/core/generic.py", line 5304, in _reindex_axes
    new_index, indexer = ax.reindex(
  File "/opt/conda/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 4434, in reindex
    target = self._wrap_reindex_result(target, indexer, preserve_names)
  File "/opt/conda/lib/python3.10/site-packages/pandas/core/indexes/multi.py", line 2556, in _wrap_reindex_result
    target = MultiIndex.from_tuples(target)
  File "/opt/conda/lib/python3.10/site-packages/pandas/core/indexes/multi.py", lin